In [10]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, confusion_matrix,roc_curve, auc
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sb
import copy
from sklearn.feature_selection import SelectKBest,chi2
from hockey_rink import BDCRink
from tqdm import tqdm
import hockey_mst
from sklearn.metrics import PrecisionRecallDisplay
import json
from sklearn.dummy import DummyClassifier
from tqdm import tqdm
import os

#display stuff
plt.rcParams["font.family"] = "Consolas"
pd.set_option('precision', 5)

In [11]:
folder_path = r'C:/Users/carli/Documents/Hockey Research/BDC/2022/weplay/Modelling Code'
file_type = '/*.csv'
files = glob.glob(folder_path + file_type)
df = pd.read_csv(max(files, key=os.path.getmtime))
print(max(files, key=os.path.getmtime))

C:/Users/carli/Documents/Hockey Research/BDC/2022/weplay/Modelling Code\df_in_plate.csv


In [12]:
print(df.columns[df.columns.str.contains("x_ft") & ~df.columns.str.contains("\.")])

Index(['away_x_ft_1', 'away_x_ft_2', 'away_x_ft_3', 'away_x_ft_4',
       'away_x_ft_5', 'away_x_ft_6', 'home_x_ft_1', 'home_x_ft_2',
       'home_x_ft_3', 'home_x_ft_4', 'home_x_ft_5', 'home_x_ft_6'],
      dtype='object')


In [13]:
## Residual Data Cleaning
df[["O Players","D Players","All MST","All_Avg_Edge","All_Total_Edge","All_Avg_Edges per Player","O MST", "O_Avg_Edge","O_Total_Edge","O_Avg_Edges_per_Player","D MST", "D_Avg_Edge","D_Total_Edge","D_Avg_Edges per Player", "OD_MST_Ratio", "All_OCR"]] = None
#df['high_danger_within_four'] = 0 #df['assumed_danger_states']
#finding columns of x and y coordinates as well as positions
x_cols = list(df.columns[df.columns.str.contains("x_ft") & ~df.columns.str.contains("\.")])
y_cols = list(df.columns[df.columns.str.contains("y_ft") & ~df.columns.str.contains("\.")])
positions = list(df.columns[df.columns.str.contains("goalie_") & ~df.columns.str.contains("\.")])

#extra variable calcuation
df_w_ind_vars = hockey_mst.ind_var_calculation(df, x_cols, y_cols, positions)

0/735
(array([ 0,  1,  7,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3      NaN
away_goalie_4      NaN
away_goalie_5      NaN
away_goalie_6      NaN
home_goalie_1     True
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5    False
home_goalie_6      NaN
Name: 0, dtype: object
Away D Players
5
1/735
(array([ 0,  1,  2,  3,  4,  6,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2     True
home_goalie_3    False
home_goalie_4    False
home_goalie_5    False
home_goalie_6      NaN
Name: 1, dtype: object
Away D Players
5
2/735
(array([ 0,  1,  2,  3,  4,  6,  7,  8, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    Fals

C:\Users\carli\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Away D Players
5
12/735
(array([ 0,  1,  2,  3,  7,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5      NaN
away_goalie_6      NaN
home_goalie_1     True
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5    False
home_goalie_6      NaN
Name: 12, dtype: object
Away D Players
5
13/735
(array([0, 1, 3, 4, 6, 7, 8], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3     True
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4      NaN
home_goalie_5      NaN
home_goalie_6      NaN
Name: 13, dtype: object
14/735
(array([ 0,  1,  2,  3,  6,  7,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5      NaN
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False


47/735
(array([0, 1, 3, 4, 6, 7, 8], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3     True
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4      NaN
home_goalie_5      NaN
home_goalie_6      NaN
Name: 47, dtype: object
48/735
(array([0, 2, 3, 4, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2     True
away_goalie_3    False
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5      NaN
home_goalie_6      NaN
Name: 48, dtype: object
49/735
(array([0, 2, 3, 4, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2     True
away_goalie_3    False
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goa

70/735
(array([0, 1, 2, 3, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5      NaN
home_goalie_6      NaN
Name: 70, dtype: object
71/735
(array([0, 1, 2, 3, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5      NaN
home_goalie_6      NaN
Name: 71, dtype: object
72/735
(array([ 0,  1,  2,  3,  6,  7,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4   

...

Away D Players
5
607/735
(array([ 0,  1,  2,  3,  4,  6,  7,  8, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4     True
home_goalie_5    False
home_goalie_6      NaN
Name: 607, dtype: object
Away D Players
5
608/735
(array([ 0,  1,  2,  3,  4,  6,  7,  8, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4     True
home_goalie_5    False
home_goalie_6      NaN
Name: 608, dtype: object
Away D Players
5
609/735
(array([ 0,  1,  2,  3,  6,  7,  8, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5      NaN
away_goalie_6      NaN
home_goal

637/735
(array([ 0,  1,  2,  3,  6,  7,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5    False
home_goalie_6      NaN
Name: 637, dtype: object
638/735
(array([ 0,  1,  2,  3,  6,  7,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5    False
home_goalie_6      NaN
Name: 638, dtype: object
639/735
(array([ 0,  1,  2,  3,  6,  7,  8,  9, 10], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goali

Name: 669, dtype: object
670/735
(array([ 0,  1,  2,  3,  6,  7,  8,  9, 10, 11], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5      NaN
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5    False
home_goalie_6    False
Name: 670, dtype: object
671/735
(array([0, 1, 2, 3, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5      NaN
home_goalie_6      NaN
Name: 671, dtype: object
672/735
(array([ 1,  2,  3,  4,  6,  7,  8,  9, 10], dtype=int64),)
away_goalie_1     True
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2   

702/735
(array([0, 1, 2, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4      NaN
away_goalie_5      NaN
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5      NaN
home_goalie_6      NaN
Name: 702, dtype: object
703/735
(array([0, 1, 2, 4, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4     True
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5      NaN
home_goalie_6      NaN
Name: 703, dtype: object
704/735
(array([0, 1, 2, 4, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4     True
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
hom

(array([0, 1, 2, 3, 6, 7, 8, 9], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4    False
away_goalie_5     True
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4    False
home_goalie_5      NaN
home_goalie_6      NaN
Name: 731, dtype: object
732/735
(array([0, 1, 2, 4, 6, 7, 8], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4     True
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3    False
home_goalie_4      NaN
home_goalie_5      NaN
home_goalie_6      NaN
Name: 732, dtype: object
733/735
(array([0, 1, 2, 4, 6, 7], dtype=int64),)
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4     True
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3      NaN
home_goalie_4      NaN
home_goalie_5    

In [14]:
df_w_ind_vars.to_csv('data_all_metrics_Nov29_final.csv')

In [15]:
print(df_w_ind_vars[["O Players","D Players"]])
print(df_w_ind_vars.loc[733,'venue'])
print(df_w_ind_vars.loc[733,df.columns.str.contains("goalie")])
print(np.mean(df_w_ind_vars[["D Players"]]))
print(np.min(df_w_ind_vars[["D Players"]]))
print(np.max(df_w_ind_vars[["D Players"]]))
print(np.mean(df_w_ind_vars[["O Players"]]))
print(np.min(df_w_ind_vars[["O Players"]]))
print(np.max(df_w_ind_vars[["O Players"]]))

    O Players D Players
0           2         5
1           5         5
2           5         5
3           5         5
4           4         5
..        ...       ...
730         5         5
731         4         5
732         3         5
733         2         5
734         2         5

[735 rows x 2 columns]
home
away_goalie_1    False
away_goalie_2    False
away_goalie_3    False
away_goalie_4     True
away_goalie_5    False
away_goalie_6      NaN
home_goalie_1    False
home_goalie_2    False
home_goalie_3      NaN
home_goalie_4      NaN
home_goalie_5      NaN
home_goalie_6      NaN
goalie             4.0
Name: 733, dtype: object
D Players    4.65881
dtype: float64
D Players    2
dtype: object
D Players    6
dtype: object
O Players    4.03745
dtype: float64
O Players    2
dtype: object
O Players    6
dtype: object


In [7]:
#correlation matrix

In [8]:
##Model Development
vars = ["high_danger_within_four","distance_to_attacking_net", "All_Avg_Edge", "All_Total_Edge","O_Avg_Edge","O_Total_Edge","O_Avg_Edges_per_Player", "D_Avg_Edge","D_Total_Edge","OD_MST_Ratio", "All_OCR"]
ind_vars = copy.deepcopy(vars) #["distance_to_attacking_net","All_Avg_Edge", "O_Avg_Edge","O_Total_Edge","O_Avg_Edges_per_Player", "D_Avg_Edge", "D_Total_Edge", "OD_MST_Ratio", "All_OCR"]
ind_vars.remove("high_danger_within_four")
#unused:
#"All_Avg_Edges per Player" because extrememly high VIF
#"D_Avg_Edges per Player" because extrememly high VIF
## of d players and # of o players because makes it worse :(

#fiddling around with the data to select the relevant dependent and independent variables in a way the relevant extraneous info can still be used when plotting on rinks
df_all_no_na = df_w_ind_vars.fillna(value=np.nan).dropna(subset = vars) #df with all columns but with rows dropped where one of the modelling variables is NaN
df_no_na = df_all_no_na[vars] #keeping only the relevant variables to modelling
# df_no_na.columns = df_no_na.columns.str.strip().str.lower()
X = df_no_na[ind_vars].reset_index().drop(columns='index') #keeping only the relevant independent variables to modelling
y = df_no_na['high_danger_within_four'] #keeping only the relevant dependent variable to modelling
X.columns = X.columns.str.strip().str.lower()

#correlation matrix
#corr_m = X.corr(method ='pearson')
#plt.figure(figsize=(40,40))
#hm = sb.heatmap(corr_m, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 25}, cmap='coolwarm', yticklabels=ind_vars, xticklabels=ind_vars)
#plt.tight_layout()

#get ratio of dep variable classes
uniques, counts = np.unique(y, return_counts=True)
scaler = counts[0]/counts[1]

#setting up interaction terms
interactions = PolynomialFeatures(interaction_only=True,include_bias = False)
X_w_inter = interactions.fit_transform(X)
new_ind_vars_raw = interactions.get_feature_names_out()

#variable selection - honestly not the best variable selection method but it's the one I can do without fucking up the data and retaining variable names to give to An
selection = SelectKBest(chi2, k=45)
trans_X = selection.fit_transform(X_w_inter, y) #k should be somewhere between 40 and 60 otherwise model is :(
selected_feature_names_raw = new_ind_vars_raw[selection.get_support()]
selected_feature_names = []
for i in selected_feature_names_raw:
    j = i.replace(' ', '_')
    selected_feature_names.append(j)

#splitting training and testing
X_train, X_test, y_train, y_test = train_test_split(trans_X, y, test_size=0.2, random_state=366)

#applying logistic model to training data
model1_log = linear_model.LogisticRegression(solver='liblinear', penalty='l1', max_iter=10000, class_weight = {0:1, 1:3.5}, random_state=43)
model1_log.fit(X_train, y_train)

#get features selected by SelectKBest and also their coefficients in the model to put in saved json object
selected_feature_names.append('intercept')
feature_dict = {key: None for key in selected_feature_names}
# key = 'All_Avg_Edge'
# key.lower()

model1_log.coef_[0]
model1_log.intercept_[0]
print(feature_dict)
m = 0
for feature in feature_dict:
    if feature == 'intercept':
        feature_dict['intercept'] = model1_log.intercept_[0]
    else:
        feature_dict[feature] = model1_log.coef_[0][m]
    m += 1
print(feature_dict)

with open("feature_dict.json", "w") as outfile:
    json.dump(feature_dict, outfile)

#old way of getting features and values
#features_coef = pd.DataFrame(np.array([selected_features, model1_log.coef_[0]]).T)
#features_coef.to_csv('2022/bdc22-mst/features_coefficients.csv', header=False, index=False)

##Model Evalution
#MSE, Score, and Confusion matrix for train and test data
dummy_clf = DummyClassifier(random_state=123, strategy='stratified')
dummy_clf.fit(X_train,y_train)
dpred = dummy_clf.predict(X_test)

# mean_squared_error(test_y, dpred)
# print("Logistic Regression Score for dummy: ", dummy_clf.score(train_y, test_y))
d = pd.DataFrame(y_test.reset_index(drop = True)).join(pd.DataFrame(dpred, columns=['pred']))
confusion_matrix(d.high_danger_within_four, d.pred)

pred1 = model1_log.predict(X_test)
mse = mean_squared_error(y_test, pred1)
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
print("Logistic Regression Score: ", model1_log.score(X_test, y_test))
print(confusion_matrix(y_test, model1_log.predict(X_test)))

pred_train = model1_log.predict(X_train)
mse = mean_squared_error(y_train, pred_train)
print("The mean squared error (MSE) on train set: {:.4f}".format(mse))
print("Logistic Regression Score: ", model1_log.score(X_train, y_train))
print(confusion_matrix(y_train, model1_log.predict(X_train)))

print(
    "\n---- MODEL RESULTS ----\n",
    "-- DUMMY CLASSIFIER --\n",
    confusion_matrix(d.high_danger_within_four, d.pred),
    "\n-- LOGISTIC REGRESSION ON TEST DATA --\n",
    "------ Confusion Matrix:\n", confusion_matrix(y_test, model1_log.predict(X_test)),
    "\nREGRESSION SCORE: ", model1_log.score(X_test, y_test),
    "\n-- LOGISTIC REGRESSION ON TRAINING DATA --\n",
    "------ Confusion Matrix:\n", confusion_matrix(y_train, model1_log.predict(X_train)),
        "\nREGRESSION SCORE: ", model1_log.score(X_train, y_train)
)

#honestly not sure what these graphs say but it makes sense to me?
#sb.kdeplot(model1_log.predict_proba(X_test)[:,1], cut=0)
#sb.kdeplot(model1_log.predict_proba(X_test)[:,0], cut=0)
#plt.show()

#ROC and PRC plot prep

#Calculate the probability scores of each point in the training set
y_train_score = model1_log.decision_function(X_train)
# Calculate the fpr, tpr, and thresholds for the training set
train_fpr, train_tpr, thresholds = roc_curve(y_train, y_train_score)
#Calculate the probability scores of each point in the test set
y_test_score = model1_log.decision_function(X_test)
#Calculate the fpr, tpr, and thresholds for the test set
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_score)

#ROC Curve
sb.lineplot(test_fpr, test_tpr, color='darkorange', lw=1, label='ROC curve', ci=False)
plt.legend(loc = 'lower right')
plt.title("AUC "+str(auc(test_fpr,test_tpr)))
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Precision Recall Curves for testing and training sets
display = PrecisionRecallDisplay.from_predictions(y_test, y_test_score, name="Log")
plt.title("Testing Set 2-class Precision-Recall curve")
plt.plot([1, 0], [0, 1],'r--')
plt.show()

display = PrecisionRecallDisplay.from_predictions(y_train, y_train_score, name="Log")
plt.title("Training Set 2-class Precision-Recall curve")
plt.plot([1, 0], [0, 1],'r--')
plt.show()

# trans_X
# y

# feats = pd.DataFrame(trans_X, columns=selected_feature_names)
# res = pd.DataFrame(model1_log.predict_proba(trans_X), columns=['no', 'yes'])

# results = df_all_no_na.reset_index(drop = True).join(res)

# combo = feats.join(res)

# raw_coord_pairs = np.array([x_coords,y_coords]).T
# raw_home_coord_pairs = raw_coord_pairs[7:][~(player_role[7:] == "Goalie")]
# home_coord_pairs = raw_home_coord_pairs[~np.isnan(raw_home_coord_pairs)]
# home_coord_pairs = home_coord_pairs.reshape(int(len(home_coord_pairs)/2),2)
# df_all_no_na['clock_seconds'].iloc[358]
# df_all_no_na["venue"].iloc[358]
# df_all_no_na["D MST"].iloc[358]
# df_all_no_na[x_cols].iloc[358]
# df_all_no_na[y_cols].iloc[360]

# df_all_no_na[positions].iloc[276]

## Plotting, still fiddling with this
for i in [278,71,37,12]:
    x_coords = df_all_no_na[x_cols].iloc[i]
    y_coords = df_all_no_na[y_cols].iloc[i]
    raw_coord_pairs = np.array([x_coords,y_coords]).T
    #print(df_all_no_na.iloc[i])
    prob = model1_log.predict_proba(np.array([trans_X[i]]))[:,1][0] #the second value in each row output by predict_proba is probability of a 1. check model1_log.classes_ for reference
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    rink = BDCRink()
    rink.draw(ax=ax)
    player_role = df_all_no_na[positions].iloc[i]
    features_coef = pd.DataFrame(np.array([np.array(selected_feature_names)[:-1], trans_X[i]]).T)

    raw_all_coord_pairs = raw_coord_pairs[~(player_role == "Goalie")]
    all_coord_pairs = raw_all_coord_pairs[~np.isnan(raw_all_coord_pairs)]
    all_coord_pairs = all_coord_pairs.reshape(int(len(all_coord_pairs)/2),2)
    plotted = np.argwhere(df_all_no_na["All MST"].iloc[i] > 0)
    for m in plotted:
        j,k = m
        rink.arrow(all_coord_pairs[j,0],all_coord_pairs[j,1],all_coord_pairs[k,0],all_coord_pairs[k,1], color= 'dimgray',width=1)

    if df_all_no_na['venue'].iloc[i] == 'home': #home team is the offensive team
        raw_home_coord_pairs = raw_coord_pairs[7:][~(player_role[7:] == "Goalie")]
        home_coord_pairs = raw_home_coord_pairs[~np.isnan(raw_home_coord_pairs)]
        home_coord_pairs = home_coord_pairs.reshape(int(len(home_coord_pairs)/2),2)

        print("Offensive")
        print(home_coord_pairs)
        plotted = np.argwhere(df_all_no_na["O MST"].iloc[i] > 0)
        for m in plotted:
            j,k = m
            rink.arrow(home_coord_pairs[j,0],home_coord_pairs[j,1],home_coord_pairs[k,0],home_coord_pairs[k,1], color= 'lightgreen')
        rink.scatter(df_all_no_na['x_coord'].iloc[i],df_all_no_na['y_coord'].iloc[i], label = "Puck Location", c='black', s=200, ax = ax,zorder=102)
        rink.scatter(home_coord_pairs.T[0],home_coord_pairs.T[1], label = "Offensive", c='lightgreen', s=160, ax = ax,edgecolors='black',zorder=101)

        raw_away_coord_pairs = raw_coord_pairs[:7]#[~(player_role[7:] == "Goalie")]
        away_coord_pairs = raw_away_coord_pairs[~np.isnan(raw_away_coord_pairs)]
        away_coord_pairs = away_coord_pairs.reshape(int(len(away_coord_pairs)/2),2)

        print("Defensive")
        print(away_coord_pairs)
        plotted = np.argwhere(df_all_no_na["D MST"].iloc[i] > 0)
        for m in plotted:
            j,k = m
            rink.arrow(away_coord_pairs[j,0],away_coord_pairs[j,1],away_coord_pairs[k,0],away_coord_pairs[k,1], color= 'darkorange')
        rink.scatter(away_coord_pairs.T[0],away_coord_pairs.T[1], label = "Defensive", c='darkorange', s=160, ax = ax,edgecolors='black',zorder=101)
    elif df_all_no_na['venue'].iloc[i] == 'away': #away is offensive team

        raw_home_coord_pairs = raw_coord_pairs[7:]#[~(player_role[7:] == "Goalie")]
        home_coord_pairs = raw_home_coord_pairs[~np.isnan(raw_home_coord_pairs)]
        home_coord_pairs = home_coord_pairs.reshape(int(len(home_coord_pairs)/2),2)
        print("Defensive")
        print(home_coord_pairs)
        plotted = np.argwhere(df_all_no_na["D MST"].iloc[i] > 0)
        for m in plotted:
            j,k = m
            rink.arrow(home_coord_pairs[j,0],home_coord_pairs[j,1],home_coord_pairs[k,0],home_coord_pairs[k,1], color= 'darkorange')
        rink.scatter(home_coord_pairs.T[0],home_coord_pairs.T[1], label = "Defensive", c='darkorange', s=160, ax=ax,edgecolors='black',zorder=101)

        raw_away_coord_pairs = raw_coord_pairs[:7][~(player_role[:7] == "Goalie")]
        away_coord_pairs = raw_away_coord_pairs[~np.isnan(raw_away_coord_pairs)]
        away_coord_pairs = away_coord_pairs.reshape(int(len(away_coord_pairs)/2),2)

        print("Offensive")
        print(away_coord_pairs)
        plotted = np.argwhere(df_all_no_na["O MST"].iloc[i] > 0)
        print(plotted)
        for m in plotted:
            j,k = m
            rink.arrow(away_coord_pairs[j,0],away_coord_pairs[j,1],away_coord_pairs[k,0],away_coord_pairs[k,1], color= 'lightgreen')
        rink.scatter(df_all_no_na['x_coord'].iloc[i],df_all_no_na['y_coord'].iloc[i], label = "Puck Location",c='black', s=200, ax=ax,zorder=102)
        rink.scatter(away_coord_pairs.T[0], away_coord_pairs.T[1], label = "Offensive", c='lightgreen', s=160,ax=ax,edgecolors='black',zorder=101)
    plt.title("Game State "+str(i)+": Probability of a Dangerous Situation = "+str(prob)+'\n # of O Players: '+str(df_all_no_na['O Players'].iloc[i])+', # of D Players (with Goalie if present): '+str(df_all_no_na['D Players'].iloc[i]), fontsize = 20)
    leg = plt.legend(fontsize = 15)
    leg.set_zorder(120)

    plt.savefig(fname='./tool_test_data/Game_State_'+str(i)+'.png')
    features_coef.to_csv('./tool_test_data/feature_values'+str(i)+'.csv', header=False, index=False)
    df_all_no_na.iloc[i].to_csv('./tool_test_data/raw_values'+str(i)+'.csv', header=False, index=True)

########################################################################################
########################################################################################
########################################################################################
########################################################################################
########################################################################################
# Loops for identifying high and low danger states and trends bt the two
# for loop for high danger states
for i in tqdm(np.arange(0, len(df_all_no_na))):
    # print('yes')
    x_coords = df_all_no_na[x_cols].iloc[i]
    y_coords = df_all_no_na[y_cols].iloc[i]
    raw_coord_pairs = np.array([x_coords,y_coords]).T
    #print(df_all_no_na.iloc[i])
    prob = model1_log.predict_proba(np.array([trans_X[i]]))[:,1][0] #the second value in each row output by predict_proba is probability of a 1. check model1_log.classes_ for reference
    # round(prob, ndigits=0.3)
    # round(1.11111111111, 1)
    prob = np.round(prob, 3)

    if prob > 0.8:
        fig, ax = plt.subplots(1, 1, figsize=(20, 10))
        rink = BDCRink()
        rink.draw(ax=ax)
        player_role = df_all_no_na[positions].iloc[i]
        features_coef = pd.DataFrame(np.array([np.array(selected_feature_names)[:-1], trans_X[i]]).T)

        raw_all_coord_pairs = raw_coord_pairs[~(player_role == "Goalie")]
        all_coord_pairs = raw_all_coord_pairs[~np.isnan(raw_all_coord_pairs)]
        all_coord_pairs = all_coord_pairs.reshape(int(len(all_coord_pairs)/2),2)
        plotted = np.argwhere(df_all_no_na["All MST"].iloc[i] > 0)
        for m in plotted:
            j,k = m
            rink.arrow(all_coord_pairs[j,0],all_coord_pairs[j,1],all_coord_pairs[k,0],all_coord_pairs[k,1], color= 'dimgray',width=1)

        if df_all_no_na['venue'].iloc[i] == 'home': #home team is the offensive team
            raw_home_coord_pairs = raw_coord_pairs[7:][~(player_role[7:] == "Goalie")]
            home_coord_pairs = raw_home_coord_pairs[~np.isnan(raw_home_coord_pairs)]
            home_coord_pairs = home_coord_pairs.reshape(int(len(home_coord_pairs)/2),2)

            # print("Offensive")
            # print(home_coord_pairs)
            plotted = np.argwhere(df_all_no_na["O MST"].iloc[i] > 0)
            for m in plotted:
                j,k = m
                rink.arrow(home_coord_pairs[j,0],home_coord_pairs[j,1],home_coord_pairs[k,0],home_coord_pairs[k,1], color= 'lightgreen')
            rink.scatter(df_all_no_na['x_coord'].iloc[i],df_all_no_na['y_coord'].iloc[i], label = "Puck Location", c='black', s=200, ax = ax,zorder=102)
            rink.scatter(home_coord_pairs.T[0],home_coord_pairs.T[1], label = "Offensive", c='lightgreen', s=160, ax = ax,edgecolors='black',zorder=101)

            raw_away_coord_pairs = raw_coord_pairs[:7]#[~(player_role[7:] == "Goalie")]
            away_coord_pairs = raw_away_coord_pairs[~np.isnan(raw_away_coord_pairs)]
            away_coord_pairs = away_coord_pairs.reshape(int(len(away_coord_pairs)/2),2)

            # print("Defensive")
            # print(away_coord_pairs)
            plotted = np.argwhere(df_all_no_na["D MST"].iloc[i] > 0)
            for m in plotted:
                j,k = m
                rink.arrow(away_coord_pairs[j,0],away_coord_pairs[j,1],away_coord_pairs[k,0],away_coord_pairs[k,1], color= 'darkorange')
            rink.scatter(away_coord_pairs.T[0],away_coord_pairs.T[1], label = "Defensive", c='darkorange', s=160, ax = ax,edgecolors='black',zorder=101)
        elif df_all_no_na['venue'].iloc[i] == 'away': #away is offensive team

            raw_home_coord_pairs = raw_coord_pairs[7:]#[~(player_role[7:] == "Goalie")]
            home_coord_pairs = raw_home_coord_pairs[~np.isnan(raw_home_coord_pairs)]
            home_coord_pairs = home_coord_pairs.reshape(int(len(home_coord_pairs)/2),2)
            # print("Defensive")
            # print(home_coord_pairs)
            plotted = np.argwhere(df_all_no_na["D MST"].iloc[i] > 0)
            for m in plotted:
                j,k = m
                rink.arrow(home_coord_pairs[j,0],home_coord_pairs[j,1],home_coord_pairs[k,0],home_coord_pairs[k,1], color= 'darkorange')
            rink.scatter(home_coord_pairs.T[0],home_coord_pairs.T[1], label = "Defensive", c='darkorange', s=160, ax=ax,edgecolors='black',zorder=101)

            raw_away_coord_pairs = raw_coord_pairs[:7][~(player_role[:7] == "Goalie")]
            away_coord_pairs = raw_away_coord_pairs[~np.isnan(raw_away_coord_pairs)]
            away_coord_pairs = away_coord_pairs.reshape(int(len(away_coord_pairs)/2),2)

            # print("Offensive")
            # print(away_coord_pairs)
            plotted = np.argwhere(df_all_no_na["O MST"].iloc[i] > 0)
            # print(plotted)
            for m in plotted:
                j,k = m
                rink.arrow(away_coord_pairs[j,0],away_coord_pairs[j,1],away_coord_pairs[k,0],away_coord_pairs[k,1], color= 'lightgreen')
            rink.scatter(df_all_no_na['x_coord'].iloc[i],df_all_no_na['y_coord'].iloc[i], label = "Puck Location",c='black', s=200, ax=ax,zorder=102)
            rink.scatter(away_coord_pairs.T[0], away_coord_pairs.T[1], label = "Offensive", c='lightgreen', s=160,ax=ax,edgecolors='black',zorder=101)
        plt.title("Game State "+str(i)+": Probability of a Dangerous Situation = "+str(prob)+'\n # of O Players: '+str(df_all_no_na['O Players'].iloc[i])+', # of D Players (with Goalie if present): '+str(df_all_no_na['D Players'].iloc[i]), fontsize = 20)
        leg = plt.legend(fontsize = 15)
        leg.set_zorder(120)

        plt.savefig(fname='./high_danger_states/plots/Game_State_'+str(i)+'.png')
        features_coef.to_csv('./high_danger_states/feature_values/feature_values'+str(i)+'.csv', header=False, index=False)
        df_all_no_na.iloc[i].to_csv('./high_danger_states/raw_values/raw_values'+str(i)+'.csv', header=False, index=True)  
    else:
        continue

# res.sort_values(by = 'yes', ascending = True).head(60)

# res[(res.no > 0.9) & (res.no < 0.95)].reset_index()

# for loop for low danger states
for i in tqdm(np.arange(0, len(df_all_no_na))):
    # print('yes')
    x_coords = df_all_no_na[x_cols].iloc[i]
    y_coords = df_all_no_na[y_cols].iloc[i]
    raw_coord_pairs = np.array([x_coords,y_coords]).T
    #print(df_all_no_na.iloc[i])
    prob = model1_log.predict_proba(np.array([trans_X[i]]))[:,1][0] #the second value in each row output by predict_proba is probability of a 1. check model1_log.classes_ for reference
    # round(prob, ndigits=0.3)
    # round(1.11111111111, 1)
    prob = np.round(prob, 3)

    # setting to take plots bt 5% and 10% chance of high danger 
    # why?
    # because...uh, there were fewer bt 5%-10% than just under 10% to comb through
    # the odds are low, but never 0
    if (prob < 0.1) & (prob > 0.05):
        fig, ax = plt.subplots(1, 1, figsize=(20, 10))
        rink = BDCRink()
        rink.draw(ax=ax)
        player_role = df_all_no_na[positions].iloc[i]
        features_coef = pd.DataFrame(np.array([np.array(selected_feature_names)[:-1], trans_X[i]]).T)

        raw_all_coord_pairs = raw_coord_pairs[~(player_role == "Goalie")]
        all_coord_pairs = raw_all_coord_pairs[~np.isnan(raw_all_coord_pairs)]
        all_coord_pairs = all_coord_pairs.reshape(int(len(all_coord_pairs)/2),2)
        plotted = np.argwhere(df_all_no_na["All MST"].iloc[i] > 0)
        for m in plotted:
            j,k = m
            rink.arrow(all_coord_pairs[j,0],all_coord_pairs[j,1],all_coord_pairs[k,0],all_coord_pairs[k,1], color= 'dimgray',width=1)

        if df_all_no_na['venue'].iloc[i] == 'home': #home team is the offensive team
            raw_home_coord_pairs = raw_coord_pairs[7:][~(player_role[7:] == "Goalie")]
            home_coord_pairs = raw_home_coord_pairs[~np.isnan(raw_home_coord_pairs)]
            home_coord_pairs = home_coord_pairs.reshape(int(len(home_coord_pairs)/2),2)

            # print("Offensive")
            # print(home_coord_pairs)
            plotted = np.argwhere(df_all_no_na["O MST"].iloc[i] > 0)
            for m in plotted:
                j,k = m
                rink.arrow(home_coord_pairs[j,0],home_coord_pairs[j,1],home_coord_pairs[k,0],home_coord_pairs[k,1], color= 'lightgreen')
            rink.scatter(df_all_no_na['x_coord'].iloc[i],df_all_no_na['y_coord'].iloc[i], label = "Puck Location", c='black', s=200, ax = ax,zorder=102)
            rink.scatter(home_coord_pairs.T[0],home_coord_pairs.T[1], label = "Offensive", c='lightgreen', s=160, ax = ax,edgecolors='black',zorder=101)

            raw_away_coord_pairs = raw_coord_pairs[:7]#[~(player_role[7:] == "Goalie")]
            away_coord_pairs = raw_away_coord_pairs[~np.isnan(raw_away_coord_pairs)]
            away_coord_pairs = away_coord_pairs.reshape(int(len(away_coord_pairs)/2),2)

            # print("Defensive")
            # print(away_coord_pairs)
            plotted = np.argwhere(df_all_no_na["D MST"].iloc[i] > 0)
            for m in plotted:
                j,k = m
                rink.arrow(away_coord_pairs[j,0],away_coord_pairs[j,1],away_coord_pairs[k,0],away_coord_pairs[k,1], color= 'darkorange')
            rink.scatter(away_coord_pairs.T[0],away_coord_pairs.T[1], label = "Defensive", c='darkorange', s=160, ax = ax,edgecolors='black',zorder=101)
        elif df_all_no_na['venue'].iloc[i] == 'away': #away is offensive team

            raw_home_coord_pairs = raw_coord_pairs[7:]#[~(player_role[7:] == "Goalie")]
            home_coord_pairs = raw_home_coord_pairs[~np.isnan(raw_home_coord_pairs)]
            home_coord_pairs = home_coord_pairs.reshape(int(len(home_coord_pairs)/2),2)
            # print("Defensive")
            # print(home_coord_pairs)
            plotted = np.argwhere(df_all_no_na["D MST"].iloc[i] > 0)
            for m in plotted:
                j,k = m
                rink.arrow(home_coord_pairs[j,0],home_coord_pairs[j,1],home_coord_pairs[k,0],home_coord_pairs[k,1], color= 'darkorange')
            rink.scatter(home_coord_pairs.T[0],home_coord_pairs.T[1], label = "Defensive", c='darkorange', s=160, ax=ax,edgecolors='black',zorder=101)

            raw_away_coord_pairs = raw_coord_pairs[:7][~(player_role[:7] == "Goalie")]
            away_coord_pairs = raw_away_coord_pairs[~np.isnan(raw_away_coord_pairs)]
            away_coord_pairs = away_coord_pairs.reshape(int(len(away_coord_pairs)/2),2)

            # print("Offensive")
            # print(away_coord_pairs)
            plotted = np.argwhere(df_all_no_na["O MST"].iloc[i] > 0)
            # print(plotted)
            for m in plotted:
                j,k = m
                rink.arrow(away_coord_pairs[j,0],away_coord_pairs[j,1],away_coord_pairs[k,0],away_coord_pairs[k,1], color= 'lightgreen')
            rink.scatter(df_all_no_na['x_coord'].iloc[i],df_all_no_na['y_coord'].iloc[i], label = "Puck Location",c='black', s=200, ax=ax,zorder=102)
            rink.scatter(away_coord_pairs.T[0], away_coord_pairs.T[1], label = "Offensive", c='lightgreen', s=160,ax=ax,edgecolors='black',zorder=101)
        plt.title("Game State "+str(i)+": Probability of a Dangerous Situation = "+str(prob)+'\n # of O Players: '+str(df_all_no_na['O Players'].iloc[i])+', # of D Players (with Goalie if present): '+str(df_all_no_na['D Players'].iloc[i]), fontsize = 20)
        leg = plt.legend(fontsize = 15)
        leg.set_zorder(120)

        plt.savefig(fname='./low_danger_states/plots/Game_State_'+str(i)+'.png')
        features_coef.to_csv('./low_danger_states/feature_values/feature_values'+str(i)+'.csv', header=False, index=False)
        df_all_no_na.iloc[i].to_csv('./low_danger_states/raw_values/raw_values'+str(i)+'.csv', header=False, index=True)  
    else:
        continue

KeyError: ['distance_to_attacking_net']